In [5]:
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import os

model = tf.keras.models.load_model("F:\\Github_Desktop\\16.MachineLearning-Project-Final\\model\\face_model.h5")
labels = os.listdir("dataset")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

attendance = pd.DataFrame(columns=["Name","Date","Time"])

attendance.loc[len(attendance)] = [
    name,
    datetime.now().strftime("%Y-%m-%d"),
    datetime.now().strftime("%H:%M:%S")
]


cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        face = gray[y:y+h,x:x+w]
        face = cv2.resize(face,(100,100))/255.0
        face = face.reshape(1,100,100,1)

        pred = model.predict(face)
        name = labels[np.argmax(pred)]

        if name not in attendance["Name"].values:
            attendance.loc[len(attendance)] = [
                name,
                datetime.now().strftime("%Y-%m-%d"),
                datetime.now().strftime("%H:%M:%S")
            ]

        cv2.putText(frame,name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

    cv2.imshow("Attendance",frame)

    if cv2.waitKey(1)==27:
        break

attendance.to_csv("attendance.csv",index=False)

cam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step
